In [1]:
import os
import cv2
import numpy as np
from keras.utils import np_utils
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import shutil

from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt

In [2]:
from keras.utils import np_utils

from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras import Model, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D, LeakyReLU, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.optimizers import Ftrl
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.nn import leaky_relu
from tensorflow.keras.applications import Xception, VGG19, InceptionV3, ResNet50V2, InceptionResNetV2
from tensorflow.keras.activations import relu
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.layers.experimental import RandomFourierFeatures
from tensorflow.keras.losses import Hinge

import os
import numpy as np
from numpy.random import seed
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot

In [ ]:
def apply_clahe(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(40, 40))
    img = clahe.apply(img)
    img = cv2.GaussianBlur(img, (3,3), 0)
    return img

In [ ]:
train_path = os.listdir("nus-cs5242/train_image/train_image")
out_path = "nus-cs5242/preprocessed/"
for index in range(len(train_path)):
    img_path = "nus-cs5242/train_image/train_image/" + train_path[index]
    img = apply_clahe(img_path)
    cv2.imwrite(out_path + train_path[index], img)

# Move Images

In [3]:
labels = pd.read_csv('nus-cs5242/train_label.csv')
image_path = "nus-cs5242/preprocessed/"
for index, value in labels.iterrows():
    image_src_path = image_path + str(value.ID) + ".png"
    image_dst_path = image_path + str(value.Label) + "/" + str(value.ID) + ".png"
    shutil.move(image_src_path, image_dst_path)

FileNotFoundError: [Errno 2] No such file or directory: 'nus-cs5242/preprocessed/0.png'

In [4]:
my_seed = 5242
img_directory = "nus-cs5242/preprocessed"
datagen = ImageDataGenerator(brightness_range=(0.3, 1.0), rescale=1./255, validation_split=0.2)
train_datagen = datagen.flow_from_directory(img_directory, target_size=(512, 512), seed=my_seed, subset='training')
validation_datagen = datagen.flow_from_directory(img_directory, target_size=(512, 512), seed=my_seed, subset='validation')

Found 933 images belonging to 3 classes.
Found 231 images belonging to 3 classes.


In [5]:
total = len(train_datagen.labels)
weight_for_0 = (1 / sum(train_datagen.labels == 0))*(total)/3.0
weight_for_1 = (1 / sum(train_datagen.labels == 1))*(total)/3.0
weight_for_2 = (1 / sum(train_datagen.labels == 2))*(total)/3.0
class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

In [6]:
base_model = Xception(weights='imagenet', input_shape=(512, 512, 3), include_top=False)
base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(Conv2D(128, (3, 3)))
# add_model.add(LeakyReLU())
add_model.add(Conv2D(128, (3, 3)))
# add_model.add(LeakyReLU())
add_model.add(Conv2D(128, (3, 3)))
add_model.add(BatchNormalization())
add_model.add(Activation('relu'))
add_model.add(MaxPooling2D(pool_size=(2,2)))

# add_model.add(Dropout(0.3))
add_model.add(Flatten())
add_model.add(RandomFourierFeatures(output_dim=128, kernel_initializer="gaussian"))
# add_model.add(Dropout(0.3))
add_model.add(Dense(3, activation = 'softmax', kernel_regularizer='l1_l2'))

model = add_model
model.compile(loss=Hinge(), optimizer=Adamax(), metrics=[CategoricalAccuracy()])

In [7]:
model.fit(train_datagen, validation_data=validation_datagen, epochs=20, class_weight=class_weight)

Epoch 1/20
30/30 [==============================] - 57s 2s/step - loss: 1.2490 - categorical_accuracy: 0.7631 - val_loss: 1.4395 - val_categorical_accuracy: 0.5195
Epoch 2/20
30/30 [==============================] - 22s 746ms/step - loss: 1.1226 - categorical_accuracy: 0.8853 - val_loss: 1.2417 - val_categorical_accuracy: 0.7835
Epoch 3/20
30/30 [==============================] - 22s 748ms/step - loss: 1.0454 - categorical_accuracy: 0.9496 - val_loss: 1.0622 - val_categorical_accuracy: 0.9177
Epoch 4/20
30/30 [==============================] - 22s 749ms/step - loss: 0.9895 - categorical_accuracy: 0.9689 - val_loss: 1.0106 - val_categorical_accuracy: 0.9177
Epoch 5/20
30/30 [==============================] - 22s 749ms/step - loss: 0.9489 - categorical_accuracy: 0.9775 - val_loss: 0.9740 - val_categorical_accuracy: 0.9307
Epoch 6/20
30/30 [==============================] - 23s 750ms/step - loss: 0.9130 - categorical_accuracy: 0.9839 - val_loss: 0.9469 - val_categorical_accuracy: 0.9437
E

In [8]:
for i in range(775, len(base_model.layers)):
    if ('batch_normalization' not in base_model.layers[i].name):
        base_model.layers[i].trainable = True

In [9]:
model.compile(optimizer=Adamax(1e-5),  # Very low learning rate
              loss='categorical_crossentropy',
              metrics=[CategoricalAccuracy()])
model.fit(train_datagen, validation_data=validation_datagen, epochs=10, class_weight=class_weight)

Epoch 1/10
30/30 [==============================] - 23s 783ms/step - loss: 0.1139 - categorical_accuracy: 1.0000 - val_loss: 0.2343 - val_categorical_accuracy: 0.9567
Epoch 2/10
30/30 [==============================] - 23s 751ms/step - loss: 0.1132 - categorical_accuracy: 1.0000 - val_loss: 0.2289 - val_categorical_accuracy: 0.9610
Epoch 3/10
30/30 [==============================] - 23s 753ms/step - loss: 0.1128 - categorical_accuracy: 1.0000 - val_loss: 0.2301 - val_categorical_accuracy: 0.9610
Epoch 4/10
30/30 [==============================] - 23s 754ms/step - loss: 0.1126 - categorical_accuracy: 1.0000 - val_loss: 0.2303 - val_categorical_accuracy: 0.9610
Epoch 5/10
30/30 [==============================] - 23s 753ms/step - loss: 0.1125 - categorical_accuracy: 1.0000 - val_loss: 0.2339 - val_categorical_accuracy: 0.9524
Epoch 6/10
30/30 [==============================] - 23s 754ms/step - loss: 0.1122 - categorical_accuracy: 1.0000 - val_loss: 0.2208 - val_categorical_accuracy: 0.965

In [10]:
model.save("model_clahe2_rff")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_clahe2_rff\assets
